# Section 1: TCAV
Please refer this source https://github.com/tensorflow/tcav on how you can import and run TCAV. The repository consists of an example (https://github.com/tensorflow/tcav/blob/master/Run_TCAV_on_colab.ipynb) of TCAV scores using GoogLeNet model trained on ImageNet dataset. The examples demonstrated the results based the concepts ["dotted","striped","zigzagged"] with target being 'zebra'. 

You are allowed to change the code template in accordance with the example provided. 

In [1]:
# !pip install tcav
# !pip install tensorflow==1.15
# !pip install keras==2.2.4

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar100, cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.applications.resnet50 import ResNet50

import sys
import os
sys.path.insert(0, os.path.abspath('../..'))

from cav.tcav import *

np.random.seed(1996)

In [11]:
def extract_airplane_set():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    airplane_train_indexes = [i for i, x in enumerate(y_train == [0]) if x]
    airplane_test_indexes = [i for i, x in enumerate(y_test == [0]) if x]
    x_train_airplane = x_train[airplane_train_indexes]
    y_train_airplane = y_train[airplane_train_indexes]
    x_test_airplane = x_test[airplane_test_indexes]
    y_test_airplane = y_test[airplane_test_indexes]

    return x_train_airplane, y_train_airplane, x_test_airplane, y_test_airplane

def extract_cloud_set():
    (x_train_concept, y_train_concept), (x_test_concept, y_test_concept) = cifar100.load_data()
    concept_train_indexes = [i for i, x in enumerate(y_train_concept == [23]) if x]
    x_train_concept_cloud = x_train_concept[concept_train_indexes]
    y_train_concept_cloud = y_train_concept[concept_train_indexes]
 
    return x_train_concept_cloud, y_train_concept_cloud

def load_and_transfer_learn():
    resnet_model = ResNet50(include_top=False, weights='imagenet', input_shape=(32, 32, 3), pooling="avg")

    # Freeze the pre-trained layers
    for layer in resnet_model.layers:
      layer.trainable = False
    
    tl_model = Sequential()
    tl_model.add(resnet_model)
    tl_model.add(Flatten())
    tl_model.add(Dense(512))
    tl_model.add(Activation('relu'))
    tl_model.add(Dropout(0.5))
    tl_model.add(Dense(1))
    tl_model.add(Activation('sigmoid'))
    return tl_model

def tcav_scores(model, x_train, y_train, x_train_concept):
    bottleneck = 3
    tf.compat.v1.disable_eager_execution()
    tcav_obj = TCAV()
    tcav_obj.set_model(model)
    tcav_obj.split_model(bottleneck = bottleneck, conv_layer = True)
    tcav_obj.train_cav(x_train_concept)
    tcav_obj.calculate_sensitivity(x_train, y_train)
    tcav_obj.print_sensitivity()



In [12]:
x_train, y_train, x_test, y_test = extract_airplane_set()

In [13]:
x_train_concept, y_train_concept = extract_cloud_set()

In [14]:
model = load_and_transfer_learn()

batch_size = 32
epochs = 5
loss_func = "binary_crossentropy"    #<--- has to be in string format
optim = "SGD"       #<--- has to be in string format

# Create a callback that saves the model's weights
model.compile(loss=loss_func, optimizer=optim, metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

Instructions for updating:
Colocations handled automatically by placer.


Train on 5000 samples, validate on 1000 samples
Epoch 1/5
4960/5000 [============================>.] - ETA: 0s - loss: 0.0306 - accuracy: 0.9942

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


5000/5000 [==============================] - 4s 767us/sample - loss: 0.0304 - accuracy: 0.9942 - val_loss: 9.3350e-08 - val_accuracy: 1.0000
Epoch 2/5
5000/5000 [==============================] - 2s 495us/sample - loss: 1.9326e-06 - accuracy: 1.0000 - val_loss: 9.3068e-08 - val_accuracy: 1.0000
Epoch 3/5
5000/5000 [==============================] - 2s 492us/sample - loss: 6.5222e-06 - accuracy: 1.0000 - val_loss: 9.2240e-08 - val_accuracy: 1.0000
Epoch 4/5
5000/5000 [==============================] - 3s 592us/sample - loss: 4.9333e-06 - accuracy: 1.0000 - val_loss: 9.1411e-08 - val_accuracy: 1.0000
Epoch 5/5
5000/5000 [==============================] - 3s 543us/sample - loss: 5.3731e-06 - accuracy: 1.0000 - val_loss: 9.0680e-08 - val_accuracy: 1.0000


In [15]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 activation_2 (Activation)   (None, 512)               0         
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 513       
                                                                 
 activation_3 (Activation)   (None, 1)                

In [16]:
tcav_scores(model, x_train, y_train, x_train_concept)

/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


The sensitivity of class 1 is  nan
The sensitivity of class 0 is  1.0


/content/cav/tcav.py:154: RuntimeWarning: invalid value encountered in long_scalars
  np.sum(self.sensitivity[np.where(self.y_labels == 1)[0]] > 0)
